In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np

Begin by setting values. All lengths are in nanometers, and angles are in degrees. 

In [ ]:
lambda_p = 775.0
omega_p = 2*math.pi/lambda_p
lambda_s = np.linspace(1500,1600,num=100)
lambda_i = np.linspace(1500,1600,num=100)
omega_s = 2*math.pi/lambda_s
omega_i = 2*math.pi/lambda_i

# Setting Sellmeier coefficients for the ordinary and extraordinary axis of the KDP crystal
KDP_o = [2.259276,13.005522,400,0.01008956,0.012942625]
KDP_e = [2.132668,3.2279924,400,0.008637494,0.012281043]

# Defining the optic axis
theta_c = 51.7
phi_s = 0

# Setting the length of the crystal in nanometers 
L = 10**5

Define a function for refractive index along a fixed axis of a material, with quantities derived from Sellmeier coefficients as parameters. 

In [ ]:
def axisIndex(wavelength, sellmeier):
    wavelength = wavelength/1000
    n_1 = sellmeier[0]
    n_2 = sellmeier[1]*(wavelength**2)/((wavelength**2)-sellmeier[2])
    n_3 = sellmeier[3]/((wavelength**2)-sellmeier[4])
    return np.sqrt(n_1+n_2+n_3)  

Below, we construct the refIndex function to calculate the fast and slow refractive indices of refraction for a photon along a given direction of travel. Here, in accordance with the KDP crystal, we assume two axes with ordinary refractive index and one with extraordinary, so $n_x=n_y=n_o$ and $n_z=n_e$. Then for a photon with direction vector $\hat{s}$, we find the refractive index using the equation

$$ \frac{s_x^2}{n^{-2}(\hat{s})-n_x^{-2}} + \frac{s_y^2}{n^{-2}(\hat{s})-n_y^{-2}} + \frac{s_z^2}{n^{-2}(\hat{s})-n_z^{-2}}. $$

We can rearrange this to obtain a quadratic equation in terms of $x=1/n^2$ of the form 

$$ x^2 - Bx + C = 0 $$

where 

$$ B = s_x^2 \left(\frac{1}{n_y^2} + \frac{1}{n_z^2}\right)+ s_y^2 \left(\frac{1}{n_x^2}+\frac{1}{n_z^2}\right) + s_z^2 \left(\frac{1}{n_x^2}+\frac{1}{n_y^2}\right) $$

and 

$$ C = \frac{s_x^2}{n_y^2n_z^2} + \frac{s_y^2}{n_x^2n_z^2} + \frac{s_z^2}{n_x^2n_y^2}. $$

In [ ]:
def refIndex(theta, phi, wavelength, fast):
    theta = math.radians(theta)
    s_x=np.sin(theta)*np.cos(phi)
    s_y=np.sin(theta)*np.sin(phi)
    s_z=np.cos(theta)
    n_x = n_y = axisIndex(wavelength,KDP_o)
    n_z = axisIndex(wavelength,KDP_e)
        
    b = (s_x**2)*((n_y**-2)+(n_z**-2))+(s_y**2)*((n_x**-2)+(n_z**-2))+(s_z**2)*((n_x**-2)+(n_y**-2))
    c = (s_x**2)*(n_y**-2)*(n_z**-2)+(s_y**2)*(n_x**-2)*(n_z**-2)+(s_z**2)*(n_x**-2)*(n_y**-2)
    if fast == True:
        x = (b+np.sqrt((b**2)-4*c))/2
    else:
        x = (b-np.sqrt((b**2)-4*c))/2
    return np.sqrt(1/x)

Here, we calculate $\Delta k$ assuming that the photons differ only in the $z$ direction and that they have the same value of $\theta$. In this case, 

$$\Delta k = n_p\omega_p - n_s\omega_s - n_i\omega_i. $$

The printout statement shows that the calculated $\Delta k$ is very low for $\lambda_s=\lambda_i=1550$ nm at the phase matching angle of $51.7^\circ$, which is what we expect. 

In [ ]:
# Defining delta k assuming deviation only in the z direction
def delta_k(omega_p,omega_s,omega_i,n_p,n_s,n_i):
    k_z = omega_p*n_p-omega_s*n_s-omega_i*n_i
    return k_z

# Testing delta_k with sample wavelengths for which phase-matching should occur 
w_s = 2*math.pi/1550
w_i = 2*math.pi/1550
n_p = refIndex(theta_c,0,lambda_p,True)
n_s = refIndex(theta_c,0,2*math.pi/w_s,False)
n_i = refIndex(theta_c,0,2*math.pi/w_i,False)

dk = delta_k(omega_p,w_s,w_i,n_p,n_s,n_i)
print(dk)

We calculate phase matching based on the $z$ component of the interaction Hamiltonian

$$ \int\limits_0^L e^{i\Delta k_z z}\ dz. $$

Evaluating this integral gives us 

$$ \frac{1}{i\Delta k_z}e^{i\Delta k_z z}\biggr\rvert_0^L = -\frac{i}{\Delta k_z}\left(e^{i\Delta k_z L}-1\right). $$

Using the double angle formulas $\cos 2x = 1 - 2\sin^2 x$ and $\sin 2x = 2\sin x\cos x$, this becomes

$$ -\frac{i}{\Delta k_z}\left(1-2\sin^2\left(\frac{1}{2}\Delta k_z L\right)+2i\sin\left(\frac{1}{2}\Delta k_z L\right)\cos\left(\frac{1}{2}\Delta k_z L\right)-1\right) $$.

Distributing the $i$ and factoring out $\sin(\Delta k_z L/2)$, we obtain 

$$\frac{1}{\Delta k_z} \left[2\sin\left(\frac{1}{2}\Delta k_z L\right)\left(\cos\left(\frac{1}{2}\Delta k_z L\right)+ i\sin\left(\frac{1}{2}\Delta k_z L\right) \right)\right]=\frac{1}{\frac{1}{2}\Delta k_z}\sin\left(\frac{1}{2}\Delta k_z L\right)e^{i\frac{1}{2}\Delta k_z L}.$$

Multiplying this by complex conjugate and dividing by $L^2$, this becomes 
$$ \frac{1}{L^2}\left(\frac{1}{\frac{1}{2}\Delta k_z}\sin\left(\frac{1}{2}\Delta k_z L\right)\right)^2 e^{i\frac{1}{2}\Delta k_z L}e^{-i\frac{1}{2}\Delta k_z L},$$
which simplifies to
$$ \left(\frac{\sin\left(\frac{1}{2}\Delta k_z L\right)}{\frac{1}{2}\Delta k_z L}\right)^2. $$

Thus, we begin below by calculating
$$\frac{\sin\left(\frac{1}{2}\Delta k_z L\right)}{\frac{1}{2}\Delta k_z L}$$.

In [ ]:
# Calculating phase matching based on delta k
def phaseMatch(dk, L):
    x = np.sin(dk*L)/(dk*L)
    return x

print(phaseMatch(dk, 1*10**6))

Below, we construct a mesh of values of the refractive indices of the signal and idler photons at various wavelengths, and then use this to calculate an array of $\Delta k$ and phase-matching values. 

In [ ]:
# Plotting the sinc function
x_mesh,y_mesh=np.meshgrid(lambda_s,lambda_i)
n_p = refIndex(theta_c,0,lambda_p, True)
nsMesh = refIndex(theta_c,phi_s,x_mesh, False)
niMesh = refIndex(theta_c,phi_s,y_mesh, False)
dk = delta_k(omega_p,2*math.pi/x_mesh,2*math.pi/y_mesh,n_p,nsMesh,niMesh)
pm = phaseMatch(dk, L)
plt.pcolormesh(x_mesh, y_mesh, pm**2, shading='nearest')
plt.show()